In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
from glob import glob
import numpy as np
from numpy import matlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from src import plotting
from scipy import ndimage
from tqdm import tqdm
import warnings
import nibabel as nib

In [2]:
process = 'Group_Analysis'
top_path = '/Users/emcmaho7/Dropbox/projects/SI_EEG/SIEEG_analysis'
input_path = f'{top_path}/data/interim'
out_path = f'{top_path}/data/interim/{process}'
figure_path = f'{top_path}/reports/figures/{process}'
Path(out_path).mkdir(parents=True, exist_ok=True)
Path(figure_path).mkdir(parents=True, exist_ok=True)

regress_gaze = False
overwrite = False

## fMRI Whole Brain

In [11]:
import nibabel as nib
import numpy as np
from nilearn import plotting, surface
import nibabel as nib
from src.tools import camera_switcher
import cv2
import os
import re
import warnings

os.environ["SUBJECTS_DIR"] = "/Users/emcmaho7/Dropbox/projects/SI_EEG/SIEEG_analysis/data/raw/freesurfer"
os.environ["FREESURFER_HOME"] = "/Applications/freesurfer"


def compute_surf_stats(prefix, sub, hemi):
    file = f'{prefix}_hemi-{hemi}.mgz'
    if not os.path.exists(file):
        cmd = '/Applications/freesurfer/bin/mri_vol2surf '
        cmd += f'--src {prefix}.nii.gz '
        cmd += f'--out {file} '
        cmd += f'--regheader sub-{sub} '
        cmd += f'--hemi {hemi} '
        cmd += '--projfrac 1 '
        cmd += '> /dev/null 2>&1'
        os.system(cmd)
    return surface.load_surf_data(file)


def load_surf_mesh(path, sub, hemi):
    return f'{path}/freesurfer/sub-{sub}/surf/{hemi}.inflated', \
            f'{path}/freesurfer/sub-{sub}/surf/{hemi}.sulc'


def plot_stats(surf_mesh, bg_map, surf_map, hemi_, figure_prefix,
               vmax=0.3, threshold=1e-6, 
               title=None, cmap_name='magma', 
               views=['ventral', 'medial', 'lateral']):
    cmap=sns.color_palette(cmap_name, as_cmap=True)
    hemi_name = 'left' if hemi_ == 'lh' else 'right'
    
    for view in views:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="choosing both vmin and a threshold is not allowed; setting vmin to 0")
            fig = plotting.plot_surf_roi(surf_mesh=surf_mesh,
                                        roi_map=surf_map,
                                        bg_map=bg_map,
                                        vmax=vmax,
                                        vmin=0., 
                                        engine='plotly',
                                        colorbar=True,
                                        view=view,
                                        cmap=cmap,
                                        title=title,
                                        title_font_size=30,
                                        hemi=hemi_name)
            fig.figure.update_layout(coloraxis_colorbar=dict(
                                    title='Explained variance ($r^2$)',
                                    tickvals=[0, vmax],  # Positions of the ticks
                                    ticktext=list(np.linspace(0, vmax, 4).round(2)),
                                    thickness=25,
                                    len=0.75,
                                    x=1.02),
                                     scene_camera=camera_switcher(hemi_, view))
            fig.figure.write_image(f'{figure_prefix}_view-{view}_hemi-{hemi_}.png')


def pngs_to_mp4(input_folder, output_file,
                file_pattern, fps=9, frame_size=None,
                start_frame=None, end_frame=None):
    """
    Converts a series of PNG images in a folder into an MP4 video file.

    Parameters:
    - input_folder: Path to the folder containing PNG images.
    - output_file: Path to save the MP4 video file.
    - fps: Frames per second in the output video.
    - frame_size: Tuple of (width, height) for the video frame size. If None, the size of the first image is used.
    """

    # Get all the PNG files in the folder
    images = glob(os.path.join(input_folder, file_pattern))
    images = sorted(images)  # Sort the images by name
    if start_frame is not None and end_frame is not None: 
        images = [image for image in images if start_frame < int(re.search("timepoint-(\d+)", image).group(1)) < end_frame]

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
    if not frame_size:
        # If frame size is not specified, use the first image to determine the size
        first_image = cv2.imread(images[0])
        frame_size = (first_image.shape[1], first_image.shape[0])
    out = cv2.VideoWriter(output_file, fourcc, fps, frame_size)

    for image_path in images:
        img = cv2.imread(image_path)
        # Resize the image to match the frame size, if necessary
        if img.shape[1] != frame_size[0] or img.shape[0] != frame_size[1]:
            img = cv2.resize(img, frame_size)
        out.write(img)

    # Release everything when the job is finished
    out.release()
    cv2.destroyAllWindows()

**Combine Whole Brain results across EEG subjects**

**THIS IS VERY SLOW**


In [4]:
whole_brain_files = glob(f'{out_path}/sub-*_whole-brain-group.csv.gz')
if len(whole_brain_files) != 4: 
    files = sorted(glob(f'{input_path}/fMRIDecoding/sub-*_reg-gaze-False_time-*_whole-brain-decoding.csv.gz'))
    df = []
    for file in tqdm(files, total=len(files), desc='Loading whole brain results'):
        df.append(pd.read_csv(file))
    df = pd.concat(df)

    df = df.groupby(['subj_id', 'time', 'voxel_id']).mean(numeric_only=True).reset_index()
    for id, time_df in tqdm(df.groupby('subj_id'), total=len(df.subj_id.unique()), desc='Saving fMRI subj'):
        time_df.to_csv(f'{out_path}/sub-{str(id).zfill(2)}_whole-brain-group.csv.gz', index=False)

In [5]:
plotting_subj = 2
start, end = -.1, .75
raw_path = f'{top_path}/data/raw'
subj_id = str(plotting_subj).zfill(2)
Path(f'{input_path}/{process}/brain_plots').mkdir(parents=True, exist_ok=True)
Path(f'{figure_path}/brain_plots').mkdir(parents=True, exist_ok=True)

time_df = pd.read_csv(f'{out_path}/sub-{str(plotting_subj).zfill(2)}_whole-brain-group.csv.gz')
time_df[['i_index', 'j_index', 'k_index']] = time_df[['i_index', 'j_index', 'k_index']].astype('int')
time_df = time_df.loc[(time_df.time > start) & (time_df.time < end)].reset_index(drop=True)

In [13]:
hemis = ['lh', 'rh']
views = ['ventral', 'medial', 'lateral']

time_iterator = tqdm(enumerate(time_df.groupby('time')), total=len(time_df.time.unique()), desc='Plotting subjects')
for i, (time, cur_df) in time_iterator:
    title = f'{time * 1000:.0f} ms' 
    time_point = str(i).zfill(3)
    stat_file = f'{input_path}/{process}/brain_plots/sub-{subj_id}_timepoint-{time_point}'
    plot_file = f'{figure_path}/brain_plots/sub-{subj_id}_timepoint-{time_point}'

    img = nib.load(f'{raw_path}/fmri_betas/sub-{subj_id}_space-T1w_desc-train-fracridge_data.nii.gz')

    dims = img.shape[:-1]
    header, affine = img.header, img.affine

    scores_arr = cur_df['r2'].to_numpy()
    scores_arr[scores_arr < 0.] = 1e-5 #Threshold at 0 for plotting
    indices = cur_df[['i_index', 'j_index', 'k_index']].to_numpy()

    score_img = np.zeros(dims)
    score_img[indices[:, 0], indices[:, 1], indices[:, 2]] = scores_arr
    score_img = nib.Nifti1Image(score_img, affine=affine, header=header)
    nib.save(score_img, f'{stat_file}.nii.gz')

    for hemi in hemis:
        surf = compute_surf_stats(stat_file, subj_id, hemi)
        inflated, sulcus = load_surf_mesh(raw_path, subj_id, hemi)
        plot_stats(inflated, sulcus, surf, hemi, plot_file,
                   title=title, cmap_name='rocket',
                   views=views, vmax=.08)

Plotting subjects: 100%|████████████████████████| 44/44 [28:37<00:00, 39.03s/it]


In [14]:
for hemi in hemis: 
    for view in views:
        output_file = f'{figure_path}/sub-{str(plotting_subj).zfill(2)}_view-{view}_hemi-{hemi}.mp4'
        file_pattern = f'sub-{str(plotting_subj).zfill(2)}*view-{view}*hemi-{hemi}*.png'
        pngs_to_mp4(input_folder=f'{figure_path}/brain_plots',
                    output_file=output_file, fps=6,
                    file_pattern=file_pattern, start_frame=6, end_frame=100)